In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy_utils import database_exists, create_database
from local_settings import postgresql as settings


In [2]:
def get_engine(user, passwd, host, port, db):
    url = f"postgresql://{user}:{passwd}@{host}:{port}/{db}"
    if not database_exists(url):
        create_database(url)
    engine = create_engine(url, pool_size=50, echo=False)
    return engine

In [3]:
engine = get_engine(settings['pguser'],
          settings['pgpasswd'],
          settings['pghost'],
          settings['Port'],
          settings['pgdb'])

In [4]:
engine.url.database

'SQL_challenge'

In [5]:
def get_engine_from_settings():
    keys = ['pguser','pgpasswd','pghost','Port','pgdb']
    if not all(key in keys for key in settings.keys()):
        raise Exception('Bad config file')
        
    return get_engine(settings['pguser'],
                      settings['pgpasswd'],
                      settings['pghost'],
                      settings['Port'],
                      settings['pgdb'])


In [6]:
def get_session():
    engine = get_engine_from_settings()
    session = sessionmaker(bind=engine)()
    return session


In [7]:
session = get_session()

In [8]:
session.close()

In [9]:
engine = session.get_bind()
engine.dispose()

In [10]:
conn = engine.connect()


In [11]:
data_emp=pd.read_sql("SELECT*FROM employees", conn)
data_emp

,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date
0,473302.0,s0001,1953-07-25,Hideyuki,Zallocco,M,1990-04-28
1,475053.0,e0002,1954-11-18,Byong,Delgrande,F,1991-09-07
2,57444.0,e0002,1958-01-30,Berry,Babb,F,1992-03-21
3,421786.0,s0001,1957-09-28,Xiong,Verhoeff,M,1987-11-26
4,282238.0,e0003,1952-10-28,Abdelkader,Baumann,F,1991-01-18
...,...,...,...,...,...,...,...
300019,464231.0,s0001,1958-08-14,Constantino,Eastman,M,1988-10-28
300020,255832.0,e0002,1955-05-08,Yuping,Dayang,F,1995-02-26
300021,76671.0,s0001,1959-06-09,Ortrud,Plessier,M,1988-02-24
300022,264920.0,s0001,1959-09-22,Percy,Samarati,F,1994-09-08


In [12]:
data_emp.to_csv("../output/employees.csv")  


In [13]:
data_basic=pd.read_sql("SELECT*FROM a_basic_info", conn)
data_basic.head(10)

,emp_no,last_name,first_name,sex,salary
0,10001.0,Facello,Georgi,M,60117.0
1,10006.0,Preusig,Anneke,F,40000.0
2,10014.0,Genin,Berni,M,46168.0
3,10017.0,Bouloucos,Cristinel,F,71380.0
4,10018.0,Peha,Kazuhide,F,55881.0
5,10031.0,Joslin,Karsten,M,40000.0
6,10040.0,Meriste,Weiyi,F,52153.0
7,10045.0,Shanbhogue,Moss,M,41971.0
8,10048.0,Syrotiuk,Florian,M,40000.0
9,10052.0,Nitsch,Heping,M,57212.0


In [14]:
data_basic.to_csv("../output/salaries.csv")  


In [18]:
data_salaries=pd.read_sql("SELECT*FROM salaries", conn)


In [19]:
data_salaries.head()

,emp_no,salary
0,10001.0,60117.0
1,10002.0,65828.0
2,10003.0,40006.0
3,10004.0,40054.0
4,10005.0,78228.0


In [17]:
data_dept=pd.read_sql("SELECT*FROM department_info", conn)

In [20]:
data_dept.head()

,dept_name,dept_no,emp_no
0,Development,d005,10001.0
1,Sales,d007,10002.0
2,Production,d004,10003.0
3,Production,d004,10004.0
4,Human Resources,d003,10005.0


In [ ]:
data_dept.to_csv("../output/dept_names.csv")  


In [21]:
data_merge=pd.merge(data_basic,data_dept, how="inner")
data_merge

,emp_no,last_name,first_name,sex,salary,dept_name,dept_no
0,10001.0,Facello,Georgi,M,60117.0,Development,d005
1,10006.0,Preusig,Anneke,F,40000.0,Development,d005
2,10014.0,Genin,Berni,M,46168.0,Development,d005
3,10017.0,Bouloucos,Cristinel,F,71380.0,Marketing,d001
4,10018.0,Peha,Kazuhide,F,55881.0,Production,d004
...,...,...,...,...,...,...,...
331598,499947.0,Koyama,Conrado,F,53325.0,Production,d004
331599,499947.0,Koyama,Conrado,F,53325.0,Customer Service,d009
331600,499966.0,Crabtree,Mihalis,F,75579.0,Sales,d007
331601,499985.0,Lukaszewicz,Gila,M,40000.0,Research,d008


In [39]:
mode=data_merge['salary'].value_counts()
mode

40000.0     105316
44564.0         21
55581.0         21
53078.0         20
45050.0         20
             ...  
75983.0          1
84173.0          1
101494.0         1
81653.0          1
100447.0         1
Name: salary, Length: 50355, dtype: int64

In [30]:
g=data_merge.groupby('dept_name').max()
g

,emp_no,last_name,first_name,sex,salary,dept_no
dept_name,,,,,,
Customer Service,499991.0,dAstous,Zvonko,M,122423.0,d009
Development,499997.0,dAstous,Zvonko,M,110449.0,d005
Finance,499998.0,dAstous,Zvonko,M,114784.0,d002
Human Resources,499992.0,dAstous,Zvonko,M,110308.0,d003
Marketing,499992.0,dAstous,Zvonko,M,118250.0,d001
Production,499999.0,dAstous,Zvonko,M,104390.0,d004
Quality Management,499964.0,dAstous,Zvonko,M,95634.0,d006
Research,499985.0,dAstous,Zvonko,M,105384.0,d008
Sales,499988.0,dAstous,Zvonko,M,129492.0,d007


In [24]:
l=round(data_merge.groupby('dept_name').mean(),2)
l

,emp_no,salary
dept_name,,
Customer Service,253580.94,47998.67
Development,252932.49,48697.33
Finance,253224.87,59533.52
Human Resources,251452.36,44678.65
Marketing,256052.98,61095.90
Production,253319.07,48760.45
Quality Management,253182.24,46456.01
Research,254026.88,48850.19
Sales,253296.60,69832.13


,emp_no,last_name,first_name,sex,salary,dept_no
dept_name,,,,,,
Customer Service,10011.0,Aamodt,Aamer,F,40000.0,d009
Development,10001.0,Aamodt,Aamer,F,40000.0,d005
Finance,10042.0,Aamodt,Aamer,F,40000.0,d002
Human Resources,10005.0,Aamodt,Aamer,F,40000.0,d003
Marketing,10017.0,Aamodt,Aamer,F,40000.0,d001
Production,10003.0,Aamodt,Aamer,F,40000.0,d004
Quality Management,10009.0,Aamodt,Aamer,F,40000.0,d006
Research,10007.0,Aamodt,Aamer,F,40000.0,d008
Sales,10002.0,Aamodt,Aamer,F,40000.0,d007
